## Récupérations des images

In [ ]:
PATH_TO_RESOURCES = "../resources"

In [ ]:
import cv2

In [ ]:
symbol_files = !ls $PATH_TO_RESOURCES/**/c*.png  | sort
symbol_files

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

In [ ]:
symbols = [load_image(symbol_file) for symbol_file in symbol_files]

## Récupération des labels

In [ ]:
import re
import numpy as np

In [ ]:
symbols_labels = np.array([re.search(".+/([0-9]{2})/.+\.png", symbol_file).group(1) for symbol_file in symbol_files])

# Transformateurs

## Extraction de la zone d'intérêt
1. Récupération des données d'entrainement
2. Calcul des masques
3. Extraction des pixels à utiliser dans chaque image

### Explications

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
# define a function to compute the mask of a bubble
def symbol2mask(symbol):
    return np.sum(symbol, axis=2) < 255 * 3

In [ ]:
symbols_masks = [symbol2mask(symbol) for symbol in symbols]

In [ ]:
plt.figure(figsize=(8,16 * 10))
for ii in range(57):
    plt.subplot(57,2,1+2*ii)
    plt.imshow(symbols[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Original")
    plt.subplot(57,2,2+2*ii)
    plt.imshow(symbols_masks[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Recolored")

### Etape de pipeline

In [ ]:
class SymbolExtractor():
    def __init__(self):
        self.masks = []
        
    def fit(self, X, y=None, sample_weight=None):
        for symbol in X:
            self.masks.append(np.sum(symbol, axis=2) < 255 * 3)
        
        return self
    
    def transform(self, X, copy=None):
        return [X[i][self.masks[i]] for i in range(len(X))]

## Réduction des couleurs
Les images sont composées de nombreuses  nuances de couleurs qui ne sont pas forcément nécessaires. On va donc chercher à réduire le nombre couleurs différentes.

In [ ]:
from sklearn.cluster import MiniBatchKMeans

### Explications

In [ ]:
sample_pixels = symbols[0][symbols_masks[0]]
for i in range(1, len(symbols)):
    sample_pixels = np.concatenate((sample_pixels, symbols[i][symbols_masks[i]]), axis=0)

sample_pixels.shape, sample_pixels.dtype

In [ ]:
def find_and_plot_cluster_centers(clusters_nb):
    # Train kmeans
    kmeans = MiniBatchKMeans(clusters_nb, random_state=42)
    kmeans.fit(sample_pixels)
    
    # Create the LUT
    color_lut = np.uint8(kmeans.cluster_centers_)
    
    # Sort it by hue
    hue_lut = cv2.cvtColor(color_lut.reshape(1,color_lut.shape[0], color_lut.shape[1]), cv2.COLOR_BGR2HSV)[:, :, 0]
    indices = hue_lut.argsort(axis=None)
    color_lut_sorted = color_lut[indices]
    
    # Plot it
    plt.bar(np.arange(len(color_lut)), 
        np.ones(len(color_lut)), 
        color=color_lut_sorted/255)
    
    return kmeans, color_lut

In [ ]:
%%time
# let us display the images
plt.figure(figsize=(8,30))
max_clusters_nb = 15
variances = np.zeros(max_clusters_nb - 1)

for ii in range(1, max_clusters_nb):
    plt.subplot(max_clusters_nb, 1, ii)
    
    kmeans, color_lut = find_and_plot_cluster_centers(ii)
    plt.title(ii)
    
    variances[ii - 1] = kmeans.inertia_

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
plt.plot(np.arange(1, max_clusters_nb), variances)
plt.xlabel("Number of clusters")
plt.ylabel("Variance")
plt.title("Variance in function of the number of clusters")
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

6 clusters semble être un bon choix.

Vérifions cette supposition.

In [ ]:
def recolor_masked_know(symbol, symbol_mask, clusterer, color_lut):
    recolored = np.ones_like(symbol) * 255
    recolored[symbol_mask] = color_lut[np.uint8(clusterer.predict(symbol[symbol_mask]))]
    return recolored

In [ ]:
%%time
# reduce the color of all bubbles
# This is exactly what we did for the full poster, but applied to each bubble AND keeping background as black.
kmeans, color_lut = find_and_plot_cluster_centers(6)
symbols_quant = [recolor_masked_know(symbols[i], symbols_masks[i], kmeans, color_lut) for i in range(len(symbols))]

In [ ]:
# let us display the images
plt.figure(figsize=(8,16 * 5))
for ii in range(20):
    plt.subplot(57,2,1+2*ii)
    plt.imshow(symbols[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Original")
    plt.subplot(57,2,2+2*ii)
    plt.imshow(symbols_quant[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Recolored")

On peut remarquer que le vert et le violet se confondent.

In [ ]:
kmeans, color_lut = find_and_plot_cluster_centers(8)
symbols_quant = [recolor_masked_know(symbols[i], symbols_masks[i], kmeans, color_lut) for i in range(len(symbols))]

Pour 8 clusters le violet apparaît.

In [ ]:
plt.figure(figsize=(8,16 * 5))
for ii in range(20):
    plt.subplot(57,2,1+2*ii)
    plt.imshow(symbols[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Original")
    plt.subplot(57,2,2+2*ii)
    plt.imshow(symbols_quant[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Recolored")

On observe des résultats plus satisfaisants qu'avec 6 clusters, on va donc en utiliser 8.

### Etape de pipeline

Pas d'étape ici. Vu qu'on se sert des centres de clusters pour réduire les couleurs, on va directement utiliser le résultat de prédiction pour l'étape de pipeline de l'histogramme. Cela permet de "sauter" la réduction tout en en conservant le principe.

# Extraction de caractéristiques

## Histogramme couleur

### Explications

In [ ]:
def symbol2hist(symbol, symbol_mask, clusterer):
    return np.bincount(np.uint8(clusterer.predict(symbol[symbol_mask])), minlength = clusterer.n_clusters) / np.count_nonzero(symbol_mask)

In [ ]:
color_histograms = [symbol2hist(symbols_quant[i], symbols_masks[i], kmeans) for i in range(len(symbols))]

In [ ]:
colors_for_bars = color_lut/255  # matplotlib colors are RGB values scaled to [0,1]
for ii in range(20):
    plt.figure(figsize=(8,4))
    plt.subplot(1,3,1)
    symbol_index = ii * 5
    plt.imshow(symbols[symbol_index])
    plt.axis('off'); plt.title("Original")
    plt.subplot(1,3,2)
    plt.imshow(symbols_quant[symbol_index])
    plt.axis('off'); plt.title("Recolored")
    plt.subplot(1,3,3, aspect=len(color_histograms[0]))
    plt.ylim(0, 1)
    plt.bar(range(len(color_histograms[symbol_index])), 
            color_histograms[symbol_index]/color_histograms[symbol_index].max(), 
            color=colors_for_bars)
    plt.xticks([]); plt.title("Histogram")
    plt.show()

### Etape de pipeline

In [ ]:
class ColorHistogram():
    def __init__(self):
        self.kmeans = MiniBatchKMeans(8)
        
    def fit(self, X, y=None, sample_weight=None):
        sample_pixels = X[0]
        for i in range(1, len(X)):
            sample_pixels = np.concatenate((sample_pixels, X[i]), axis=0)
        # Train kmeans
        self.kmeans.fit(sample_pixels)
        
        return self
    
    def transform(self, X, copy=None):
        histograms = []
        for i in range(len(X)):
            histo = np.bincount(np.uint8(self.kmeans.predict(X[i])),
                        minlength = self.kmeans.n_clusters) / len(X[i])
            histograms.append(histo)

        return histograms

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([("masking", SymbolExtractor()), ("histogram", ColorHistogram()), ("clf", LinearSVC())], verbose=True)
pipeline.fit(symbols, symbols_labels)
pipeline.score(symbols, symbols_labels)